In [1]:
import os
os.chdir('../..')

In [2]:
from FLF.TorchFederatedLearnerCIFAR100 import TorchFederatedLearnerCIFAR100Config

In [3]:
conf = TorchFederatedLearnerCIFAR100Config()

In [6]:
project_name = "10-clients"

server_lr = 0.01
client_lr = 0.01
server_opt = "Adam"
client_opt = "SGD"
client_opt_strategy = "reinit"

# image_norm = "tflike"
max_rounds = 30
C = 1
NC = 10
E = 1
B = 20
is_iid = False

def get_args(opt):
    if opt == "Adam":
        return {"betas": (0.0, 0.99), "eps": 0.01}
    elif opt == "SGD":
        return {"momentum": 0.9}
    else:
        return {}
    
config = TorchFederatedLearnerCIFAR100Config(
    BREAK_ROUND=300,
    CLIENT_LEARNING_RATE=client_lr,
    CLIENT_OPT=client_opt,
    # CLIENT_OPT_L2=1e-4,
    CLIENT_OPT_STRATEGY=client_opt_strategy,
    SERVER_OPT=server_opt,
    SERVER_OPT_ARGS=get_args(server_opt),
    SERVER_LEARNING_RATE=server_lr,
    IS_IID_DATA=is_iid,
    BATCH_SIZE=B,
    CLIENT_FRACTION=C,
    N_CLIENTS=NC,
    N_EPOCH_PER_CLIENT=E,
    MAX_ROUNDS=max_rounds,
    DL_N_WORKER=0,
    NORM="group",
    # IMAGE_NORM=image_norm,
    INIT="keras",
)

{'TARGET_ACC': None,
 'BREAK_ROUND': 300,
 'MAX_ROUNDS': 30,
 'N_CLIENTS': 10,
 'CLIENT_FRACTION': 1.0,
 'N_EPOCH_PER_CLIENT': 1,
 'BATCH_SIZE': 20,
 'CLIENT_LEARNING_RATE': 0.01,
 'SERVER_LEARNING_RATE': 0.01,
 'DL_N_WORKER': 0,
 'SEED': None,
 'CLIENT_OPT': 'SGD',
 'CLIENT_OPT_L2': 0,
 'CLIENT_OPT_STRATEGY': 'reinit',
 'SERVER_OPT': 'Adam',
 'IS_IID_DATA': False,
 'IMAGE_NORM': 'thlike',
 'NORM': 'group',
 'INIT': 'keras',
 'SERVER_OPT_BETAS_0': 0.0,
 'SERVER_OPT_BETAS_1': 0.99,
 'SERVER_OPT_EPS': 0.01}

In [38]:
import comet_ml
from comet_ml.query import Parameter
import functools
import operator

comet_api = comet_ml.api.API()
exps = comet_api.query('federated-learning', project_name, functools.reduce(operator.and_, [Parameter(k) == v if not isinstance(v, bool) else Parameter(k) == str(v) for k, v in config.flatten().items() if not k.endswith('LEARNING_RATE')]))

In [43]:
def get_df(exps):
    def get_list(t, c):
        res = []
        for exp in exps:
            response = c(exp)
            if isinstance(response, list):
                res.append(None)
            else:
                res.append(t(response['valueMax']))
        return res
    def get_params(t, n):
        return get_list(t, lambda exp: exp.get_parameters_summary(n))
    def get_metrics(t, n):
        return get_list(t, lambda exp: exp.get_metrics_summary(n))

    ids = [exp.id for exp in exps]

    c_lr = get_params(float, 'CLIENT_LEARNING_RATE')
    s_lr = get_params(float, 'SERVER_LEARNING_RATE')

    teas = get_metrics(float, 'test_acc')
    return pd.DataFrame({'acc': teas, 
                         'c_lr': c_lr, 
                         's_lr': s_lr,
                            })

In [44]:
get_df(exps)

,acc,c_lr,s_lr
0,None,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
5,False,False,False
6,False,False,False
7,False,False,False
8,False,False,False
9,False,False,False


In [46]:
float('0.001')

0.001

In [36]:
query = []
for i, (k, v) in enumerate(config.flatten().items()):
#     if i > 14:
#         break
    if not k.endswith('LEARNING_RATE'):
        print(k)
        query.append(Parameter(k) == v if not isinstance(v, bool) else Parameter(k) == str(v))
exps = comet_api.query('federated-learning', project_name, functools.reduce(operator.and_, query))
exps

TARGET_ACC
BREAK_ROUND
MAX_ROUNDS
N_CLIENTS
CLIENT_FRACTION
N_EPOCH_PER_CLIENT
BATCH_SIZE
DL_N_WORKER
SEED
CLIENT_OPT
CLIENT_OPT_L2
CLIENT_OPT_STRATEGY
SERVER_OPT
IS_IID_DATA
IMAGE_NORM
NORM
INIT
SERVER_OPT_BETAS_0
SERVER_OPT_BETAS_1
SERVER_OPT_EPS


[<APIExperiment 'federated-learning/10-clients/a3bae498b9824f8686dbea2d37269667'>,
 <APIExperiment 'federated-learning/10-clients/c7ca4cfc2c7a4933989c89775096d8ff'>,
 <APIExperiment 'federated-learning/10-clients/a50eb01bf50c4297ae9fff3f418311f9'>,
 <APIExperiment 'federated-learning/10-clients/36e9ef20765048ce9e09a49efe31a903'>,
 <APIExperiment 'federated-learning/10-clients/663208f204e04d6990bcd510896a88d1'>,
 <APIExperiment 'federated-learning/10-clients/c014ad2581214fdd8ea682f3e12605f4'>,
 <APIExperiment 'federated-learning/10-clients/fed75ed0a12b447ea38b727820295972'>,
 <APIExperiment 'federated-learning/10-clients/50cdf5ba652f4520a5484d3abe1e06ee'>,
 <APIExperiment 'federated-learning/10-clients/f40fb7919eba4c298bedf6a5664e6b42'>,
 <APIExperiment 'federated-learning/10-clients/61c3940921504649a6404bc8d14d5bdd'>]

In [33]:
str(False)

'False'

In [48]:
exps[0].get_parameters_summary('CLIENT_LEARNING_RATE')

{'name': 'CLIENT_LEARNING_RATE',
 'valueMax': '0.01',
 'valueMin': '0.01',
 'valueCurrent': '0.01',
 'timestampMax': 1597570807093,
 'timestampMin': 1597570807093,
 'timestampCurrent': 1597570807093,
 'editable': False}